# Coconut Disease Detection Model v2
## MobileNetV2 + Focal Loss for Leaf Disease Classification

**Author:** Research Team  
**Date:** January 2026  
**Dataset:** Coconut Leaf Disease Dataset (4 classes)  

### Objectives:
- Train a deep learning model to classify coconut leaf diseases
- Handle class imbalance using Focal Loss
- Achieve balanced precision, recall, and F1-score across all classes
- Prevent overfitting using proper regularization techniques

## 1. Import Libraries and Check Environment

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"NumPy Version: {np.__version__}")

TensorFlow Version: 2.20.0
GPU Available: []
NumPy Version: 1.26.4


## 2. Configuration and Hyperparameters

In [2]:
# Paths
BASE_DIR = r'D:\SLIIT\Reaserch Project\CoconutHealthMonitor\Research\ml'
DATA_DIR = os.path.join(BASE_DIR, 'data', 'raw', 'stage_2_split')
MODEL_DIR = os.path.join(BASE_DIR, 'models', 'disease_detection_v2')

# Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
PHASE1_EPOCHS = 10  # Frozen base training
PHASE2_EPOCHS = 40  # Fine-tuning
INITIAL_LR = 0.001
DROPOUT_RATE = 0.4
LABEL_SMOOTHING = 0.1
FOCAL_GAMMA = 2.0
FOCAL_ALPHA = 0.25

# Create model directory
os.makedirs(MODEL_DIR, exist_ok=True)

print("="*70)
print("CONFIGURATION")
print("="*70)
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Phase 1 Epochs: {PHASE1_EPOCHS}")
print(f"Phase 2 Epochs: {PHASE2_EPOCHS}")
print(f"Initial Learning Rate: {INITIAL_LR}")
print(f"Dropout Rate: {DROPOUT_RATE}")
print(f"Label Smoothing: {LABEL_SMOOTHING}")
print(f"Focal Loss Gamma: {FOCAL_GAMMA}")

CONFIGURATION
Image Size: 224x224
Batch Size: 32
Phase 1 Epochs: 10
Phase 2 Epochs: 40
Initial Learning Rate: 0.001
Dropout Rate: 0.4
Label Smoothing: 0.1
Focal Loss Gamma: 2.0


## 3. Define Focal Loss

Focal Loss helps handle class imbalance by down-weighting easy examples and focusing on hard ones.

In [3]:
class FocalLoss(tf.keras.losses.Loss):
    """Focal Loss for handling class imbalance.
    
    Focal Loss adds a modulating factor (1-p)^gamma to cross-entropy loss,
    focusing learning on hard misclassified examples.
    
    Args:
        gamma: Focusing parameter. Higher values focus more on hard examples.
        alpha: Weighting factor for the classes.
        label_smoothing: Label smoothing factor for regularization.
    """
    def __init__(self, gamma=2.0, alpha=0.25, label_smoothing=0.0, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.alpha = alpha
        self.label_smoothing = label_smoothing
    
    def call(self, y_true, y_pred):
        # Apply label smoothing
        if self.label_smoothing > 0:
            num_classes = tf.cast(tf.shape(y_true)[-1], tf.float32)
            y_true = y_true * (1.0 - self.label_smoothing) + (self.label_smoothing / num_classes)
        
        # Clip predictions to prevent log(0)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        
        # Calculate cross entropy
        cross_entropy = -y_true * tf.math.log(y_pred)
        
        # Calculate focal weight
        weight = self.alpha * y_true * tf.pow(1 - y_pred, self.gamma)
        
        # Apply focal weight to cross entropy
        focal_loss = weight * cross_entropy
        
        return tf.reduce_sum(focal_loss, axis=-1)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'gamma': self.gamma,
            'alpha': self.alpha,
            'label_smoothing': self.label_smoothing
        })
        return config

print("Focal Loss configured with:")
print(f"  - Gamma: {FOCAL_GAMMA} (focus on hard examples)")
print(f"  - Alpha: {FOCAL_ALPHA} (class weight factor)")
print(f"  - Label Smoothing: {LABEL_SMOOTHING}")

Focal Loss configured with:
  - Gamma: 2.0 (focus on hard examples)
  - Alpha: 0.25 (class weight factor)
  - Label Smoothing: 0.1


## 4. Load and Explore Dataset

In [4]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# No augmentation for validation/test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'val'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'test'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get class information
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)

print(f"\n{'='*70}")
print("DATASET SUMMARY")
print("="*70)
print(f"Classes: {class_names}")
print(f"Number of classes: {num_classes}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")

Found 24998 images belonging to 4 classes.
Found 838 images belonging to 4 classes.
Found 837 images belonging to 4 classes.

DATASET SUMMARY
Classes: ['Leaf Rot', 'Leaf_Spot', 'healthy', 'not_cocount']
Number of classes: 4
Training samples: 24998
Validation samples: 838
Test samples: 837


## 5. Visualize Sample Images

In [5]:
# Visualize sample images from each class
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Sample Images from Each Class', fontsize=16, fontweight='bold')

for idx, class_name in enumerate(class_names):
    class_dir = os.path.join(DATA_DIR, 'train', class_name)
    sample_images = os.listdir(class_dir)[:2]
    
    for j, img_name in enumerate(sample_images):
        img_path = os.path.join(class_dir, img_name)
        img = plt.imread(img_path)
        row = j
        col = idx
        axes[row, col].imshow(img)
        axes[row, col].set_title(class_name, fontsize=12)
        axes[row, col].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'sample_images.png'), dpi=150, bbox_inches='tight')
plt.close()
print("Sample images visualization saved to: sample_images.png")

Sample images visualization saved to: sample_images.png


## 6. Calculate Class Weights

In [6]:
# Calculate class weights to handle imbalance
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights_array))

print("="*70)
print("CLASS WEIGHTS")
print("="*70)
print("Class weights (to handle imbalance):")
for i, name in enumerate(class_names):
    print(f"  {name}: {class_weights[i]:.4f}")

CLASS WEIGHTS
Class weights (to handle imbalance):
  Leaf Rot: 1.2499
  Leaf_Spot: 1.2499
  healthy: 1.2499
  not_cocount: 0.6251


## 7. Build Model Architecture

Using MobileNetV2 as the base model with a custom classification head.

In [7]:
def build_model(num_classes, dropout_rate=0.4):
    """Build MobileNetV2 model with custom classification head."""
    
    # Load pre-trained MobileNetV2 (without top layers)
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Build custom classification head
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    return model, base_model

# Build model
model, base_model = build_model(num_classes, DROPOUT_RATE)

print("="*70)
print("MODEL ARCHITECTURE - MobileNetV2")
print("="*70)
print(f"Model: MobileNetV2 + Custom Head")
print(f"Total params: {model.count_params():,}")
trainable = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
non_trainable = sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights])
print(f"Trainable params: {trainable:,}")
print(f"Non-trainable params: {non_trainable:,}")

MODEL ARCHITECTURE - MobileNetV2
Model: MobileNetV2 + Custom Head
Total params: 2,625,476
Trainable params: 363,012
Non-trainable params: 2,262,464


## 8. Phase 1: Train with Frozen Base

In [8]:
# Compile model for Phase 1
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss=FocalLoss(gamma=FOCAL_GAMMA, alpha=FOCAL_ALPHA, label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy']
)

# Callbacks for Phase 1
callbacks_phase1 = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("="*70)
print("PHASE 1: TRAINING WITH FROZEN BASE")
print("="*70)
print("Training classifier head only (base frozen)...\n")

# Train Phase 1
history_phase1 = model.fit(
    train_generator,
    epochs=PHASE1_EPOCHS,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks_phase1,
    verbose=2
)

print(f"\nPhase 1 Complete!")
print(f"Best Validation Accuracy: {max(history_phase1.history['val_accuracy'])*100:.2f}%")

PHASE 1: TRAINING WITH FROZEN BASE
Training classifier head only (base frozen)...

Epoch 1/10
782/782 - 562s - accuracy: 0.8846 - loss: 0.0370 - val_accuracy: 0.9558 - val_loss: 0.0164 - lr: 0.0010
Epoch 2/10
782/782 - 521s - accuracy: 0.9331 - loss: 0.0190 - val_accuracy: 0.9463 - val_loss: 0.0159 - lr: 0.0010
Epoch 3/10
782/782 - 517s - accuracy: 0.9423 - loss: 0.0155 - val_accuracy: 0.9582 - val_loss: 0.0131 - lr: 0.0010
Epoch 4/10
782/782 - 518s - accuracy: 0.9488 - loss: 0.0144 - val_accuracy: 0.9642 - val_loss: 0.0115 - lr: 0.0010
Epoch 5/10
782/782 - 530s - accuracy: 0.9570 - loss: 0.0122 - val_accuracy: 0.9678 - val_loss: 0.0121 - lr: 0.0010
Epoch 6/10
782/782 - 522s - accuracy: 0.9584 - loss: 0.0122 - val_accuracy: 0.9606 - val_loss: 0.0121 - lr: 0.0010
Epoch 7/10
782/782 - 519s - accuracy: 0.9607 - loss: 0.0111 - val_accuracy: 0.9594 - val_loss: 0.0117 - lr: 0.0010
Epoch 8/10
782/782 - 516s - accuracy: 0.9643 - loss: 0.0105 - val_accuracy: 0.9618 - val_loss: 0.0121 - lr: 0.00

## 9. Phase 2: Fine-tuning

Unfreeze top layers of base model for fine-tuning.

In [9]:
# Unfreeze top layers for fine-tuning
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=INITIAL_LR/10),
    loss=FocalLoss(gamma=FOCAL_GAMMA, alpha=FOCAL_ALPHA, label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy']
)

# Callbacks for Phase 2
callbacks_phase2 = [
    ModelCheckpoint(
        os.path.join(MODEL_DIR, 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("="*70)
print("PHASE 2: FINE-TUNING")
print("="*70)
print(f"Unfreezing top 50 layers for fine-tuning...")
print(f"Total layers: {len(base_model.layers)}")
print(f"Trainable layers: {len(base_model.layers) - fine_tune_at}\n")

# Train Phase 2
history_phase2 = model.fit(
    train_generator,
    epochs=PHASE2_EPOCHS,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks_phase2,
    verbose=2
)

print(f"\nPhase 2 Complete!")
print(f"Best Validation Accuracy: {max(history_phase2.history['val_accuracy'])*100:.2f}%")

PHASE 2: FINE-TUNING
Unfreezing top 50 layers for fine-tuning...
Total layers: 156
Trainable layers: 50

Epoch 1/40
782/782 - 598s - accuracy: 0.9758 - loss: 0.0073 - val_accuracy: 0.9857 - val_loss: 0.0055 - lr: 1.0000e-04
Epoch 2/40
782/782 - 595s - accuracy: 0.9832 - loss: 0.0054 - val_accuracy: 0.9845 - val_loss: 0.0052 - lr: 1.0000e-04
Epoch 3/40
782/782 - 596s - accuracy: 0.9864 - loss: 0.0046 - val_accuracy: 0.9869 - val_loss: 0.0044 - lr: 1.0000e-04
Epoch 4/40
782/782 - 598s - accuracy: 0.9888 - loss: 0.0040 - val_accuracy: 0.9881 - val_loss: 0.0044 - lr: 1.0000e-04
Epoch 5/40
782/782 - 601s - accuracy: 0.9897 - loss: 0.0036 - val_accuracy: 0.9905 - val_loss: 0.0040 - lr: 1.0000e-04
Epoch 6/40
782/782 - 599s - accuracy: 0.9905 - loss: 0.0034 - val_accuracy: 0.9869 - val_loss: 0.0044 - lr: 1.0000e-04
Epoch 7/40
782/782 - 602s - accuracy: 0.9917 - loss: 0.0030 - val_accuracy: 0.9869 - val_loss: 0.0045 - lr: 1.0000e-04
Epoch 8/40
782/782 - 604s - accuracy: 0.9921 - loss: 0.0029 - 

## 10. Evaluate on Test Set

In [10]:
# Evaluate on test set
print("="*70)
print("EVALUATION ON TEST SET")
print("="*70)

test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

EVALUATION ON TEST SET

Test Accuracy: 98.69%
Test Loss: 0.0039


## 11. Classification Report (Class-wise Metrics)

In [11]:
# Get predictions
test_generator.reset()
predictions = model.predict(test_generator, verbose=0)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Classification Report
print("="*70)
print("CLASSIFICATION REPORT (Class-wise Metrics)")
print("="*70)
print()
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

# Detailed metrics
report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

print("\n" + "="*70)
print("DETAILED CLASS-WISE METRICS")
print("="*70)
print(f"{'Class':<16} {'Precision':>10} {'Recall':>10} {'F1-Score':>10} {'Support':>10}")
print("-"*63)
for class_name in class_names:
    metrics = report_dict[class_name]
    print(f"{class_name:<16} {metrics['precision']:>10.4f} {metrics['recall']:>10.4f} {metrics['f1-score']:>10.4f} {int(metrics['support']):>10}")
print("-"*63)
print(f"{'Macro Avg':<16} {report_dict['macro avg']['precision']:>10.4f} {report_dict['macro avg']['recall']:>10.4f} {report_dict['macro avg']['f1-score']:>10.4f}")

# Summary
macro_f1 = report_dict['macro avg']['f1-score']
print(f"\n{'='*70}")
print("SUMMARY METRICS")
print("="*70)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Macro F1-Score: {macro_f1*100:.2f}%")
print(f"Difference (Acc - F1): {abs(test_accuracy - macro_f1)*100:.2f}%")

CLASSIFICATION REPORT (Class-wise Metrics)

              precision    recall  f1-score   support

    Leaf Rot     1.0000    0.9760    0.9879       167
   Leaf_Spot     0.9702    1.0000    0.9849       163
     healthy     0.9605    0.9481    0.9542        77
 not_cocount     0.9930    0.9930    0.9930       430

    accuracy                         0.9869       837
   macro avg     0.9809    0.9793    0.9800       837
weighted avg     0.9870    0.9869    0.9868       837


DETAILED CLASS-WISE METRICS
Class            Precision     Recall   F1-Score    Support
---------------------------------------------------------------
Leaf Rot            1.0000     0.9760     0.9879        167
Leaf_Spot           0.9702     1.0000     0.9849        163
healthy             0.9605     0.9481     0.9542         77
not_cocount         0.9930     0.9930     0.9930        430
---------------------------------------------------------------
Macro Avg           0.9809     0.9793     0.9800

SUMMARY METRIC

## 12. Confusion Matrix

In [12]:
# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Disease Detection v2\nTest Accuracy: {:.2f}%'.format(test_accuracy*100), 
          fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'confusion_matrix.png'), dpi=150, bbox_inches='tight')
plt.close()
print("Confusion matrix saved to: confusion_matrix.png")

Confusion matrix saved to: confusion_matrix.png


## 13. Training History Visualization

In [13]:
# Combine histories
combined_history = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(combined_history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(combined_history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].axvline(x=len(history_phase1.history['accuracy'])-1, color='r', linestyle='--', 
                label='Fine-tuning Start', alpha=0.7)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0.8, 1.0])

# Loss plot
axes[1].plot(combined_history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(combined_history['val_loss'], label='Val Loss', linewidth=2)
axes[1].axvline(x=len(history_phase1.history['loss'])-1, color='r', linestyle='--', 
                label='Fine-tuning Start', alpha=0.7)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Training History - Disease Detection v2', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'training_history.png'), dpi=150, bbox_inches='tight')
plt.close()
print("Training history saved to: training_history.png")

Training history saved to: training_history.png


## 14. Save Model Information

In [14]:
# Save model info
model_info = {
    'model_name': 'disease_detection_v2',
    'base_model': 'MobileNetV2',
    'version': '2.0',
    'date_trained': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'input_size': [IMG_SIZE, IMG_SIZE, 3],
    'num_classes': num_classes,
    'classes': class_names,
    'hyperparameters': {
        'batch_size': BATCH_SIZE,
        'phase1_epochs': PHASE1_EPOCHS,
        'phase2_epochs': PHASE2_EPOCHS,
        'initial_lr': INITIAL_LR,
        'dropout_rate': DROPOUT_RATE,
        'label_smoothing': LABEL_SMOOTHING,
        'focal_gamma': FOCAL_GAMMA,
        'focal_alpha': FOCAL_ALPHA
    },
    'dataset': {
        'train_samples': train_generator.samples,
        'val_samples': val_generator.samples,
        'test_samples': test_generator.samples
    },
    'metrics': {
        'test_accuracy': float(test_accuracy),
        'test_loss': float(test_loss),
        'macro_precision': float(report_dict['macro avg']['precision']),
        'macro_recall': float(report_dict['macro avg']['recall']),
        'macro_f1': float(report_dict['macro avg']['f1-score'])
    },
    'class_metrics': {
        name: {
            'precision': float(report_dict[name]['precision']),
            'recall': float(report_dict[name]['recall']),
            'f1_score': float(report_dict[name]['f1-score']),
            'support': int(report_dict[name]['support'])
        }
        for name in class_names
    }
}

with open(os.path.join(MODEL_DIR, 'model_info.json'), 'w') as f:
    json.dump(model_info, f, indent=2)

print("="*70)
print("MODEL INFORMATION SAVED")
print("="*70)
print("Model info saved to: model_info.json")

MODEL INFORMATION SAVED
Model info saved to: model_info.json


## 15. Final Summary

In [15]:
print("="*70)
print("                    TRAINING COMPLETE!")
print("="*70)
print(f"\nModel saved to: {MODEL_DIR}")

print(f"\n{'='*70}")
print("                    FINAL RESULTS")
print("="*70)
print(f"\n  Test Accuracy:    {test_accuracy*100:.2f}%")
print(f"  Macro F1-Score:   {macro_f1*100:.2f}%")
print(f"  Macro Precision:  {report_dict['macro avg']['precision']*100:.2f}%")
print(f"  Macro Recall:     {report_dict['macro avg']['recall']*100:.2f}%")

print(f"\n{'='*70}")
print("                 CLASS-WISE F1-SCORES")
print("="*70)
for name in class_names:
    print(f"\n  {name}:" + " "*(15-len(name)) + f"{report_dict[name]['f1-score']*100:.2f}%")

print(f"\n{'='*70}")
print("                    FILES GENERATED")
print("="*70)
print("\n  - best_model.keras")
print("  - model_info.json")
print("  - confusion_matrix.png")
print("  - training_history.png")
print("  - sample_images.png")
print("\n" + "="*70)

                    TRAINING COMPLETE!

Model saved to: D:\SLIIT\Reaserch Project\CoconutHealthMonitor\Research\ml\models\disease_detection_v2

                    FINAL RESULTS

  Test Accuracy:    98.69%
  Macro F1-Score:   98.00%
  Macro Precision:  98.09%
  Macro Recall:     97.93%

                 CLASS-WISE F1-SCORES

  Leaf Rot:         98.79%
  Leaf_Spot:        98.49%
  healthy:          95.42%
  not_cocount:      99.30%

                    FILES GENERATED

  - best_model.keras
  - model_info.json
  - confusion_matrix.png
  - training_history.png
  - sample_images.png

